In [ ]:
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON
import datetime, xlrd
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
import time
import urllib.request
import urllib, urllib.parse
import json
from datetime import datetime, timedelta
import os
import urllib.parse
import csv
import unicodedata
import io
import time
import numpy as np 

lastfm_public_key = "079a7d64ea52c358ad4f0afbe2f900b3"
  
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return np.unique(x) 

def if_user_exists(list1 , usr):
    if usr in list1:
        return True
    return False
    
def lovedTrack_getusers():
    file_path = '..//Lastfmlovedtracks.csv'
    line_number = 2 
    list = []
    #list.append(lastfm_username)
    
    ifile  = open(file_path, "r")
    read = csv.reader(ifile)
    for row in read :
        user = row[6] 
        list.append(user)
    return unique(list)
    
def lovedTrack_getList(lastfm_username,i):
    lovedTrack(lastfm_username)
    friends_list = []
    friends_list.append(lastfm_username)
    if i > 12000:
        return 0
    else:  
        url2 = "http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user="+lastfm_username+ "&api_key=" + lastfm_public_key + "&format=json"
        data2 = urllib.request.urlopen(url2).read().decode()
        obj = json.loads(data2)
        with open('E://jsondata.txt', 'w') as outfile:
            json.dump(obj, outfile)
            
        with open('E://jsondata.txt') as json_file:
            data = json.load(json_file)
            for p in data['friends']['user']:
                try:
                    friend_name = str(p['name'])
                    count = friends_list.count(friend_name)
                    if count > 0:
                        continue
                    else:
                        i+=1
                        friends_list.append(friend_name)
                        return 0
                except Exception as e:
                    print(e)
                    continue

            for friend in friends_list:
                try:
                    time.sleep(2)
                    lovedTrack_getList(friend,i)
                except Exception as e:
                    print(e)
                    continue
        return 0

def lovedTrack(lastfm_username):
    #exists_users = lovedTrack_getusers()
    #if if_user_exists(exists_users,lastfm_username) == True:
    #    return 0
    #else: 
    # build URL
    url = "http://ws.audioscrobbler.com/2.0/?method=user.getlovedtracks&user="+ lastfm_username + "&api_key=" + lastfm_public_key + "&format=json"

    data = urllib.request.urlopen(url).read().decode()
    obj = json.loads(data)
    file_path = '..//Lastfmlovedtracks.csv'
    tracks_data = open(file_path, '+a' , newline='', encoding='utf-8')
    csvwriter = csv.writer(tracks_data)

    if os.stat(file_path).st_size == 0:
        csvwriter.writerow(["track_name", "track_mbid","listeners","playcount", "artist_name", "artist_mbid","username"])

    #print(obj)
    with open('E://jsondata.txt', "w", encoding="utf-8") as outfile:
        json.dump(obj, outfile)

    with open('E://jsondata.txt') as json_file:
        data = json.load(json_file)
        for p in data['lovedtracks']['track']:
            artist_name = str(p['artist']['name'])
            track_name = str(p['name'])

            try:
                url1 ="http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key=" + lastfm_public_key + "&artist=" + urllib.parse.quote(artist_name) + "&track=" + urllib.parse.quote(track_name) + "&format=json"
                data1 = urllib.request.urlopen(url1).read().decode()
                obj = json.loads(data1)


                listeners = obj['track']['listeners']
                playcount = obj['track']['playcount']

                csvwriter.writerow([str(p['name']), str(p['mbid']),
                                    str(listeners), str(playcount),
                                    str(p['artist']['name']),
                                    str(p['artist']['mbid']),str(lastfm_username)])

            except Exception as e:
                print(e)
                listeners = 0
                playcount = 0
                continue

            #------------------------load loved track info from Spotify----------------------
            #track_info = get_track_data_from_spotify(str(p['name']),str(p['artist']['name']))
            #----------------------------------------------------------------------------------

    tracks_data.close()
    return  0

In [ ]:
import sys
lovedTrack_getList("Schrollum" ,0)